In [8]:
import numpy as np
import pandas as pd
clust =  pd.read_csv('address_clust.csv')
stats = pd.read_csv('address_stats.csv')

In [9]:
stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [10]:
clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [11]:
stats = stats.merge(clust,  how='left', on='address_id')
stats['cluster_id'].fillna(0, inplace=True)


In [12]:
for transact in stats['transaction_id'].unique():
    COUNT = len(stats[stats['transaction_id']==transact]['cluster_id'].unique())
    stats[stats['transaction_id']==transact]['count_cluster'] = COUNT

C:\Users\spectrmen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:

stats = stats.merge(stats.groupby('transaction_id')['cluster_id'].nunique().reset_index(name='count_claster'),  how='left', on='transaction_id')

In [67]:
def sum_across_two_clusters(clust_1, clust_2):
    transactions = []
    for transaction in stats['transaction_id'].unique():
        if stats[stats['transaction_id']==transaction]['cluster_id'].unique()[0] == clust_1:
            transactions.append(transaction)
    result = stats[(stats['transaction_id'].isin(transactions)) & (stats['cluster_id']==clust_1)]['sent'].sum() - \
             stats[(stats['transaction_id'].isin(transactions)) & (stats['cluster_id']!=clust_2)]['received'].sum()
    return result/10.0**8

In [68]:
def count_fee(clust):
    transactions = []
    for transaction in stats['transaction_id'].unique():
        if stats[stats['transaction_id']==transaction]['cluster_id'].unique()[0] == clust:
            transactions.append(transaction)
    result = stats[(stats['transaction_id'].isin(transactions)) & (stats['cluster_id']==clust)]['sent'].sum() - \
             stats[(stats['transaction_id'].isin(transactions))]['received'].sum()
    return result/10.0**8

# из первый

In [69]:
print(sum_across_two_clusters(1.0, 2.0), sum_across_two_clusters(1.0, 0.0))

129.910384 301.22107288


# из второго 

In [70]:
print(sum_across_two_clusters(2.0, 1.0), sum_across_two_clusters(2.0, 0.0))

136.56566722 5.61550722


# из нулевого

In [71]:
print(sum_across_two_clusters(0.0, 1.0), sum_across_two_clusters(0.0, 2.0))

298.07128772 10.69174245


# Подсчет fee 

In [72]:
print(count_fee(1.0), count_fee(2.0), count_fee(0.0))

0.667144 0.74566722 0.95160384
